In [ ]:
# Celda 1
# Importaciones estándar
import os
import time
import logging
from datetime import datetime

# Librerías de terceros
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException

# Importaciones internas del proyecto
from src.utils import configurar_logs, formatear_mensaje, validar_numero, cargar_excel, normalizar_nombres_columnas
from src.whatsapp import NotificarMensajeWhatsApp

# Configurar logs solo si este archivo es el principal
if __name__ == "__main__":
    configurar_logs(log_file="whatsapp_message_sender.log", log_to_console=True)

# Ruta base del proyecto
base_path = os.getcwd()

# Crear una instancia de la clase para enviar mensajes por WhatsApp
# whatsapp = NotificarMensajeWhatsApp()

In [ ]:
# Celda 2
# Nombres de los archivos originales
archivos_excel = {
    "directorio": os.path.join(base_path, "directorio.xlsx"),
    "data": os.path.join(base_path, "data.xlsx")
}

# Cargar los archivos Excel
archivos_excel = {nombre: cargar_excel(ruta) for nombre, ruta in archivos_excel.items()}

# Acceder a los DataFrames directamente
df_directorio = archivos_excel["directorio"]
df_data_estaciones = archivos_excel["data"]

In [ ]:
# Celda 3
# Normalizar nombres de columnas en todos los DataFrames
df1 = normalizar_nombres_columnas(df_directorio)
df2 = normalizar_nombres_columnas(df_data_estaciones)

# Verificar resultado
print("Columnas normalizadas de df1:", df1.columns)
print("Columnas normalizadas de df2:", df2.columns)

In [ ]:
# Celda 4
# Unir los dos DataFrames en un solo tablero usando 'Estacion' como clave
df_unido = pd.merge(df1, df2, on="id_pc", how="inner")

# Mostrar las primeras filas del DataFrame unido
print("DataFrame unido:")
display(df_unido.head())


In [ ]:
# Celda 5
# Reemplazar valores vacíos con NaN
df_unido["promedio_de_cloro__mg_l_"] = df_unido["promedio_de_cloro__mg_l_"].replace("", pd.NA)

# Convertir a float
df_unido["promedio_de_cloro__mg_l_"] = pd.to_numeric(df_unido["promedio_de_cloro__mg_l_"], errors="coerce")

# Redondear los valores a 4 decimales
df_unido["promedio_de_cloro__mg_l_"] = df_unido["promedio_de_cloro__mg_l_"].round(4)

# Definir condiciones para la columna 'estado'
condiciones = [
    df_unido["promedio_de_cloro__mg_l_"].isna(),  # Si es NaN → "inactivo"
    df_unido["promedio_de_cloro__mg_l_"] < 0.5,  # Si es menor a 0.5 → "inadecuado"
    df_unido["promedio_de_cloro__mg_l_"] >= 0.5  # Si es mayor o igual a 0.5 → "adecuado"
]

# Definir valores
valores = np.array(["inactivo", "inadecuado", "adecuado"], dtype=object)

# Verificación
display(df_unido.head())


In [ ]:
# Celda 6
# Obtener la hora actual en formato HHMM (ej. 0920 o 1000)
hora_actual = datetime.now().strftime('%H%M')
hora_actual = datetime.strptime('10:00', '%H:%M').strftime('%H%M')

print("Hora_actual (HHMM):", hora_actual)

# Definir la carpeta de salida
output_dir = os.path.join(base_path, "data_procesada")
os.makedirs(output_dir, exist_ok=True)  # Crear carpeta si no existe

# Nombre del archivo con formato HHMM
nombre_archivo = f"data_filtrada_{hora_actual}.xlsx"
ruta_archivo = os.path.join(output_dir, nombre_archivo)

# Guardar el DataFrame en Excel
df_unido.to_excel(ruta_archivo, index=False)

print(f"✅ Archivo guardado en: {ruta_archivo}")


In [ ]:
# Filtrar por horario de estacion
# Definir la función parse_horario correctamente
def parse_horario(horario):
    """Convierte el horario en formato 'h:mm - h:mm' en un tuple de objetos time."""
    if pd.isna(horario) or not isinstance(horario, str):
        return None, None
    try:
        inicio, fin = horario.split(" - ")
        return (datetime.strptime(inicio.strip(), "%H:%M").time(),
                datetime.strptime(fin.strip(), "%H:%M").time())
    except ValueError:
        return None, None

# Función para verificar si la hora actual está dentro del rango
def esta_en_rango(hora_actual, horario):
    """Verifica si la hora actual está dentro del rango de un horario."""
    if horario is None or horario == (None, None):
        return False
    inicio, fin = horario  # Ya es una tupla (inicio, fin)
    return inicio <= hora_actual <= fin

# Obtener la hora actual en formato hh:mm
hora_actual = datetime.now().replace(second=0, microsecond=0).time()
print("hora_actual (hh:mm):", hora_actual)

# Columnas de horarios a evaluar
columnas_requeridas = ['primer_horario', 'segundo_horario', 'tercer_horario']

# Convertir los horarios a formato datetime.time
df_unido[columnas_requeridas] = df_unido[columnas_requeridas].applymap(parse_horario)

# Crear la nueva columna 'estado_horario'
df_unido["estado_horario"] = df_unido.apply(
    lambda row: "ACTIVO" if any(esta_en_rango(hora_actual, row[col]) for col in columnas_requeridas)
    else "INACTIVO", axis=1
)

# Mostrar los resultados
display(df_unido)


In [ ]:
def filtrar_horario_de_estaciones_por_estado(df):
    """
    Devuelve un DataFrame con solo las filas
    donde Estado sea 'Inadecuado' o 'Inactivo'.
    """

    # Lista de estados a filtrar (normalizados en minúsculas)
    estados_filtrados = ["activo"]

    # Filtrar las filas donde 'estado' sea "activo"
    df_filtrado = df[df["estado_horario"].isin(estados_filtrados)].copy()

    return df_filtrado

# Aplicamos la función de filtrado
df_filtrado = filtrar_horario_de_estaciones_por_estado(df_unido)

# Mostramos el resultado
print("Estaciones filtradas (Mal / Inactivo):\n")
display(df_filtrado)

In [ ]:
def filtrar_estaciones_por_estado(df):
    """
    Devuelve un DataFrame con solo las filas
    donde Estado sea 'Inadecuado' o 'Inactivo'.
    """

    # Limpiar y normalizar la columna 'estado'
    df["estado"] = df["estado"].fillna("").str.strip().str.lower()

    # Lista de estados a filtrar (normalizados en minúsculas)
    estados_filtrados = ["inadecuado", "inactivo"]

    # Filtrar las filas donde 'estado' sea "mal" o "inactivo"
    df_filtrado = df[df["estado"].isin(estados_filtrados)].copy()

    return df_filtrado

# Aplicamos la función de filtrado
df_filtrado = filtrar_estaciones_por_estado(df_filtrado)

# Mostramos el resultado
print("Estaciones filtradas (Mal / Inactivo):\n")
display(df_filtrado)


In [ ]:
def generar_mensaje_alerta(telefono, estado, estacion, ultima_fecha_registrada, hora_inferior, hora_superior, nivel_cloro):
    """
    Genera el mensaje de alerta según el estado y la hora de corte.
    """
    header = "Previo cordial saludo,"
    footer = "Este es un mensaje de prueba, por favor no responder."

    if estado == "inadecuado":
        body = (
            f"Se comunica que hoy {ultima_fecha_registrada}, la estación {estacion} registra un nivel de cloro inadecuado, "
            f"con un promedio de {nivel_cloro} mg/L (menor a 0,5 mg/L) entre las {hora_inferior} y {hora_superior} horas."
        )
        # recommendations = ["Revisar el sistema de dosificación de cloro", "Enviar equipo técnico a la estación"]
    elif estado == "inactivo":
        body = (
            f"Se comunica que hoy {ultima_fecha_registrada}, la estación {estacion} se encuentra en estado {estado}, "
            f"entre las {hora_inferior} y {hora_superior} horas."
        )
        # recommendations = ["Activar el monitoreo remoto", "Contactar al equipo de mantenimiento"]
    else:
        return None

    return format_message(header, body, footer)

# Generar mensajes
mensajes_de_prueba = []
for idx, row in df_filtrado.iterrows():
    msg = generar_mensaje_alerta(
        telefono=row["telefono"],
        estado=row["estado"],
        estacion=row["estacion"],
        ultima_fecha_registrada=row["última fecha registrada"],
        hora_inferior=row["hora inferior"],
        hora_superior=row["hora superior"],
        nivel_cloro=row["promedio de cloro (mg/l)"]
    )
    mensajes_de_prueba.append((row["telefono"], msg))

# Imprimir mensajes generados
for tel, texto in mensajes_de_prueba:
    print(f"Teléfono: {tel}")
    print(f"Mensaje: {texto}")
    print("------")


In [ ]:
# Validar números de teléfono antes de enviar mensajes
telefonos_validos = []
mensajes_validos = []

for tel, msg in mensajes_de_prueba:
    if validate_phone_number(tel):
        telefonos_validos.append(tel)
        mensajes_validos.append(msg)

print(f"Números válidos: {len(telefonos_validos)}")
print(f"Números inválidos: {len(mensajes_de_prueba) - len(telefonos_validos)}")


In [ ]:
# Inicializar el enviador de mensajes
sender = EnhancedWhatsAppMessageSender()

# 1. Inicializar la sesión de WhatsApp Web (solo una vez)
if sender.initialize_session():
    print("Sesión de WhatsApp Web inicializada correctamente.")
else:
    print("Error al inicializar la sesión de WhatsApp Web.")

# 2. Enviar mensajes en lote
try:
    # Enviar todos los mensajes en lote
    sender.send_batch_messages(telefonos_validos, mensajes_validos)

    # Mostrar números fallidos si los hay
    if sender.failed_numbers:
        print("\nNúmeros que fallaron:")
        for tel, error in sender.failed_numbers:
            print(f"{tel}: {error}")
            # Guardar en una variable la lista de mensajes que fallaron y luego volver a enviar los mensajes
finally:
    # Limpiar la sesión
    sender.cleanup()
